<div style="display: flex; align-items: center;">
    <img alt="udeA logo" height="140px" src="https://github.com/freddyduitama/images/blob/master/logo.png?raw=true" style="width:107px;height:152px; margin-right: 40px;">
    <div style="text-align: center;">
        <h1><b><font color='0B5345'>Sistema Predictor de grupos relacionados por el diagnóstico</font></b></h1>
        <h2><font color='0B5345'>Luisa Enciso - María Ostos - Santiago Rivera</font></h2>
    </div>
</div>


In [5]:
import pandas as pd
import numpy as np
import unicodedata

<hr>
<h2>Inspección de la base de datos</h2>

In [4]:
# Ruta del archivo Excel
archivo_excel = 'DataBase.xlsx'
# Cargar el archivo Excel
df = pd.read_excel(archivo_excel, engine='openpyxl')
# Mostrar las primeras filas
df.head()

,Num caso,Aseguradora -Código-,Aseguradora -Descripción-,Edad,Grupo Edad,Sexo,Codigo de ciudad,Fecha de ingreso,Tipo de ingreso,Fecha de egreso,...,Proc28,Proc29,Proc30,Tipo servicio,Causa Basica de muerte,Infecciones,Infección Quirurgica,GRD -Código,GRD -Descripción,Tipo GRD
0,413799.0,204,Aseguradora_29,38,35-39,F,5001,30052005,URGENCIA,19062005,...,NaN,NaN,NaN,URGENCIA ADULTOS,NaN,NO,NO,14143,MH ACCIDENTE VASCULAR CEREBRAL CON INFARTO w/MCC,M
1,412359.0,313,Aseguradora_56,27,25-29,M,5088,11032005,URGENCIA,10042005,...,NaN,NaN,NaN,NO APLICA,NaN,NO,NO,184153,MH INFECCIÓN POR VIRUS DE INMUNODEFICIENCIA HU...,M
2,412552.0,313,Aseguradora_56,39,35-39,M,5001,20032005,URGENCIA,10042005,...,NaN,NaN,NaN,URGENCIA ADULTOS,NaN,NO,NO,184153,MH INFECCIÓN POR VIRUS DE INMUNODEFICIENCIA HU...,M
3,414131.0,227,Aseguradora_46,24,20-24,M,5001,20062005,URGENCIA,22062005,...,NaN,NaN,NaN,URGENCIA ADULTOS,NaN,NO,NO,184153,MH INFECCIÓN POR VIRUS DE INMUNODEFICIENCIA HU...,M
4,410573.0,227,Aseguradora_46,60,60-64,M,5001,24112004,URGENCIA,7012005,...,NaN,NaN,NaN,URGENCIA ADULTOS,NaN,NO,NO,184112,MH INFECCIONES POST OPERATORIAS Y POSTRAUMÁTIC...,M


<hr>
<h2>Limpieza de la base de datos</h2>

In [ ]:
# Función para quitar tildes y convertir a mayúsculas
def limpiar_texto(texto):
    if isinstance(texto, str):
        texto = texto.upper()
        texto = unicodedata.normalize('NFKD', texto)
        texto = ''.join([c for c in texto if not unicodedata.combining(c)])
        return texto
    return texto

# Aplicar limpieza a todo el DataFrame
df = df.applymap(limpiar_texto)
# Limpiar tildes y poner en mayúsculas los nombres de las columnas
df.columns = [limpiar_texto(col) for col in df.columns]

# Información de NaN por columna
print('--- Valores NaN por columna ---')
print(df.isna().sum())

# Información de valores únicos por columna
descripcion_columnas = []
for col in df.columns:
    valores_unicos = df[col].nunique(dropna=True)
    muestra_valores = df[col].unique()[:5]
    descripcion_columnas.append({
        'columna': col,
        'valores_unicos': valores_unicos,
        'muestra_valores': muestra_valores
    })

print('\n--- Resumen de columnas ---')
for desc in descripcion_columnas:
    print(f"Columna: {desc['columna']}")
    print(f"  Valores únicos: {desc['valores_unicos']}")
    print(f"  Ejemplo de valores: {desc['muestra_valores']}")
    if desc['valores_unicos'] == 1:
        print('  -> ¡OJO! Solo hay un valor único, probablemente no es significativa para el modelo.')
    print()

# Sugerencia de columnas poco útiles para predecir en triage (basado en valores únicos)
print('--- Sugerencia de columnas poco útiles para predecir en triage ---')
for desc in descripcion_columnas:
    if desc['valores_unicos'] == 1:
        print(f"{desc['columna']} (solo un valor)")

# Diccionarios de normalización para columnas relevantes
normalizaciones = {
    'Tipo de ingreso': {
        'URGENCIAS': 'URGENCIA',
        'URGENCIA': 'URGENCIA',
        'URGENCIA ADULTOS': 'URGENCIA',
        'U.ADULTOS': 'URGENCIA',
        'U. ADULTOS': 'URGENCIA',
        'URGENCIA PEDIATRICA': 'URGENCIA',
        'URGENCIAS PEDIATRICAS': 'URGENCIA',
        'URGENCIA GINECOLOGICA': 'URGENCIA',
        'URGENCIAS GINECOLOGICAS': 'URGENCIA',
        'URGENCIA OBSTETRICA': 'URGENCIA',
        'URGENCIAS OBSTETRICAS': 'URGENCIA',
    },
    'Situacion al alta': {
        'MUERTO': 'FALLECIDO',
        'FALLECIDO': 'FALLECIDO',
        'DEFUNCION': 'FALLECIDO',
    },
    'ServicioAlta': {
        'URGENCIAS': 'URGENCIA',
        'URGENCIA': 'URGENCIA',
        'U.ADULTOS': 'URGENCIA',
        'U. ADULTOS': 'URGENCIA',
        'URGENCIA ADULTOS': 'URGENCIA',
    }
}

# Normalizar valores equivalentes
for columna, reemplazos in normalizaciones.items():
    if columna in df.columns:
        df[columna] = df[columna].replace(reemplazos)

# Eliminar columnas con más del 70% de NaN
umbral_nan = 0.7
porcentaje_nan = df.isna().mean()
columnas_a_eliminar = porcentaje_nan[porcentaje_nan > umbral_nan].index.tolist()

# Imprimir desglose de columnas eliminadas por exceso de NaN
for col in columnas_a_eliminar:
    cantidad_nan = df[col].isna().sum()
    print(f"Columna '{col}' eliminada por exceso de NaN: {cantidad_nan} valores NaN")

df = df.drop(columns=columnas_a_eliminar)
print(f"Columnas eliminadas por exceso de NaN (>70%): {columnas_a_eliminar}")

# Reemplazar NaN: 0 en numéricos, 'DESCONOCIDO' en el resto
for col in df.columns:
    if pd.api.types.is_numeric_dtype(df[col]):
        df[col] = df[col].fillna(0)
    else:
        df[col] = df[col].fillna('DESCONOCIDO')

C:\Users\luisa\AppData\Local\Temp\ipykernel_20316\567977153.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(limpiar_texto)


--- Valores NaN por columna ---
NUM CASO                            0
ASEGURADORA -CODIGO-                0
ASEGURADORA -DESCRIPCION-           0
EDAD                                0
GRUPO EDAD                          0
SEXO                                0
CODIGO DE CIUDAD                    0
FECHA DE INGRESO                    0
TIPO DE INGRESO                     0
FECHA DE EGRESO                     0
DIAS ESTANCIA                       0
SERVICIOALTA                        0
CUIDADOS INTENSIVOS                 0
DIAS DE UNIDAD CUIDADO INTENSIVO    0
DX DE INGRESO                       0
DX PRINCIPAL DE EGRESO              0
DX PRINCIPAL DE EGRESO .1           0
DX PPAL 3 CARACTERES                0
DXR 1                               0
DXR 2                               0
DXR 3                               0
DXR 4                               0
DXR 5                               0
DXR-6                               0
SITUACION AL ALTA                   0
PROC1             

**Guardar Base de datos organizada**

DESCRIBIR QUE SE ORGANIZO

In [9]:
# Guardar CSV con datos organizados y NaN reemplazados
csv_organizado = 'DataBaseOrganizado.csv'
df.to_csv(csv_organizado, index=False, sep=';')
print(f"CSV con datos organizados y NaN reemplazados guardado como: {csv_organizado}")

CSV con datos organizados y NaN reemplazados guardado como: DataBaseOrganizado.csv


**ESCRIBIR PORQUE ELIMINAR ESTAS CARACTERISTICAS**

In [ ]:
# Eliminar columnas poco importantes para el modelo (también en el DataFrame final)
columnas_poco_importantes = [
    'NUM CASO', 'ASEGURADORA -CODIGO-', 'ASEGURADORA -DESCRIPCION-', 'CODIGO DE CIUDAD',
    'FECHA DE INGRESO', 'FECHA DE EGRESO', 'CODIGO CAUSA EXTERNA', 'CAUSA EXTERNA'
]
columnas_poco_importantes = [col for col in columnas_poco_importantes if col in df.columns]
df_final = df.drop(columns=columnas_poco_importantes)
if columnas_poco_importantes:
    print(f"(Final) Columnas poco importantes eliminadas: {columnas_poco_importantes}")

# Eliminar columnas con solo 1 dato único y reportar cuántos datos se eliminaron
columnas_un_valor = [col for col in df_final.columns if df_final[col].nunique(dropna=True) == 1]
datos_borrados = {}
for col in columnas_un_valor:
    datos_borrados[col] = df_final.shape[0]
df_final = df_final.drop(columns=columnas_un_valor)

for col, cantidad in datos_borrados.items():
    print(f"Columna '{col}' eliminada por tener solo 1 dato único. Se eliminaron {cantidad} datos.")

# Guardar CSV con las columnas finales
csv_final = 'DataBaseFinal.csv'
df_final.to_csv(csv_final, index=False, sep=';')
print(f"CSV con columnas finales guardado como: {csv_final}") 
df_final.head()

CSV con columnas finales guardado como: DataBaseFinal.csv


,EDAD,GRUPO EDAD,SEXO,TIPO DE INGRESO,DIAS ESTANCIA,SERVICIOALTA,CUIDADOS INTENSIVOS,DIAS DE UNIDAD CUIDADO INTENSIVO,DX DE INGRESO,DX PRINCIPAL DE EGRESO,...,SITUACION AL ALTA,PROC1,PROC2,PROC3,TIPO SERVICIO,INFECCIONES,INFECCION QUIRURGICA,GRD -CODIGO,GRD -DESCRIPCION,TIPO GRD
0,38,35-39,F,URGENCIA,20,29,NO,0,2,I635,...,ALTA MEDICA,43.11,54.91,87.44,URGENCIA ADULTOS,NO,NO,14143,MH ACCIDENTE VASCULAR CEREBRAL CON INFARTO W/MCC,M
1,27,25-29,M,URGENCIA,30,156,NO,0,2,B232,...,ALTA MEDICA,87.44,0.00,0.00,NO APLICA,NO,NO,184153,MH INFECCION POR VIRUS DE INMUNODEFICIENCIA HU...,M
2,39,35-39,M,URGENCIA,21,39,NO,0,2,B227,...,ALTA MEDICA,38.93,45.23,87.44,URGENCIA ADULTOS,NO,NO,184153,MH INFECCION POR VIRUS DE INMUNODEFICIENCIA HU...,M
3,24,20-24,M,URGENCIA,2,20,NO,0,2,B208,...,ALTA MEDICA,88.01,0.00,0.00,URGENCIA ADULTOS,NO,NO,184153,MH INFECCION POR VIRUS DE INMUNODEFICIENCIA HU...,M
4,60,60-64,M,URGENCIA,44,24,NO,0,2,T814,...,ALTA MEDICA,45.16,87.44,88.79,URGENCIA ADULTOS,NO,NO,184112,MH INFECCIONES POST OPERATORIAS Y POSTRAUMATIC...,M


<hr>
METRICAS ESTADISTICAS PARA LA BASE DE DATOS

<hr>
<h1> Árbol de decisión </h1>

<hr>
<h1> Random Forest </h1>

<hr>
<h1> OTRO ALGORITMO </h1>

<hr>
<h1> Comparación de los algoritmos </h1>

<hr>
<h2> Conclusiones </h2>